In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [42]:
import pandas as pd
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 10000)
pd.set_option('display.width', 10000)
pd.set_option('max_colwidth', 10000)
import numpy as np
import string
import transformers
from transformers import BertTokenizer, BertConfig
from transformers import BertForTokenClassification, AdamW

In [3]:
df = pd.read_csv("../../data/conll03/train.txt", sep=" ")
#df = pd.read_csv("../../data/conll03/dev.txt", sep=" ")
#df = pd.read_csv("data/test.txt", sep=" ")

In [4]:
df.head()

,-DOCSTART-,-X-,-X-.1,O
0,EU,NNP,B-NP,B-ORG
1,rejects,VBZ,B-VP,O
2,German,JJ,B-NP,B-MISC
3,call,NN,I-NP,O
4,to,TO,B-VP,O


In [5]:
df = df[df['O'].notna()]

In [6]:
df.drop(['-X-', '-X-.1'], axis=1, inplace=True)

In [7]:
df.columns = ['Word', 'Tag']

In [8]:
df.head()

,Word,Tag
0,EU,B-ORG
1,rejects,O
2,German,B-MISC
3,call,O
4,to,O


In [9]:
#df.drop(df[df['Word'] == "-DOCSTART-"].index, axis=0, inplace=True)

In [10]:
df['Tag'].unique()

array(['B-ORG', 'O', 'B-MISC', 'B-PER', 'I-PER', 'B-LOC', 'I-ORG',
       'I-MISC', 'I-LOC'], dtype=object)

In [11]:
tags = df['Tag'].values
words = df['Word'].values

In [12]:
new_tags = []
for t in tags:
    if t == "B-ORG":
        new_tags.append("ORG")
    elif t == "B-PER":
        new_tags.append("PER")
    elif t == "I-PER":
        new_tags.append("PER")
    elif t == "B-LOC":
        new_tags.append("LOC")
    elif t == "I-ORG":
        new_tags.append("ORG")
    elif t == "I-LOC":
        new_tags.append("LOC")
    else:
        new_tags.append("O")

In [13]:
df.drop(['Tag'], axis=1, inplace=True)
df.drop(['Word'], axis=1, inplace=True)

In [14]:
df['Tag'] = new_tags
df['Word'] = words

In [15]:
len(df)

202388

In [16]:
sentence_no = 0
sentence = []
for w in words:
    sentence.append(sentence_no)
    if w == ".":
        sentence_no = sentence_no + 1

In [17]:
df.insert(0, 'Sentence #', sentence)

In [18]:
df.head()

,Sentence #,Tag,Word
0,0,ORG,EU
1,0,O,rejects
2,0,O,German
3,0,O,call
4,0,O,to


In [19]:
g_test = df.groupby("Sentence #")
x = pd.DataFrame({"Sentence": g_test.apply(lambda sdf: " ".join(map(str,sdf.Word))),
                       "Tag": g_test.apply(lambda sdf: ",".join(map(str,sdf.Tag)))})

In [20]:
x.head()

,Sentence,Tag
Sentence #,,
0,EU rejects German call to boycott British lamb .,"ORG,O,O,O,O,O,O,O,O"
1,Peter Blackburn BRUSSELS 1996-08-22 The European Commission said on Thursday it disagreed with German advice to consumers to shun British lamb until scientists determine whether mad cow disease can be transmitted to sheep .,"PER,PER,LOC,O,O,ORG,ORG,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
2,Germany 's representative to the European Union 's veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer .,"LOC,O,O,O,O,ORG,ORG,O,O,O,PER,PER,O,O,O,O,O,O,O,O,O,O,O,LOC,O,O,O,O,O,O,O"
3,"We do n't support any such recommendation because we do n't see any grounds for it , the Commission 's chief spokesman Nikolaus van der Pas told a news briefing .","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,ORG,O,O,O,PER,PER,PER,PER,O,O,O,O,O"
4,He said further scientific study was required and if it was found that action was needed it should be taken by the European Union .,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,ORG,ORG,O"


In [21]:
import re

def decontracted(phrase):
    # general
    phrase = re.sub(r" \'s", "\'s", phrase)
    phrase = re.sub(r" \'t", "\'t", phrase)
    phrase = re.sub(r" n't", "n't", phrase)
    phrase = re.sub(r" \'re", "\'re", phrase)
    phrase = re.sub(r" \'d", "\'d", phrase)
    phrase = re.sub(r" \'ll", "\'ll", phrase)
    phrase = re.sub(r" \'t", "\'t", phrase)
    phrase = re.sub(r" \'ve", "\'ve", phrase)
    phrase = re.sub(r" \'m", "\'m", phrase)
    return phrase

In [22]:
import string
sentence = "We do n't support any such recommendation because we do n't see any grounds for it , the Commission 's chief spokesman Nikolaus van der Pas told a news briefing ."
tags = "O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,ORG,O,O,O,PER,PER,PER,PER,O,O,O,O,O"
words = sentence.split(" ")
ind = [ i for i, word in enumerate(words) if word == "'s" or word == "'t" or word == "'re" or word == "'d" or word == "'ll" or word == "'t" or word == "'ve" or word == "'m" or word == "n't"]
print(ind)
print(len(words))
labels = tags.split(",")
print(len(labels))
sentence = " ".join(words)
#print(sentence)
sentence = decontracted(sentence)
#print(sentence)
tags = [ elem for i, elem in enumerate(labels) if i not in ind]
print(len(tags))
#words = sentence.split()
#tags = [ elem for i, elem in enumerate(labels) if i not in ind]
import string
stripped = sentence.translate(str.maketrans('', '', string.punctuation))
stripped = stripped.split(" ")
#stripped = [w.translate(table) for w in words]
#stripped = [x for x in stripped if x]
print(stripped)
print(tags)
print(len(stripped))
print(len(tags))

[2, 10, 19]
31
31
28
['We', 'dont', 'support', 'any', 'such', 'recommendation', 'because', 'we', 'dont', 'see', 'any', 'grounds', 'for', 'it', '', 'the', 'Commissions', 'chief', 'spokesman', 'Nikolaus', 'van', 'der', 'Pas', 'told', 'a', 'news', 'briefing', '']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'ORG', 'O', 'O', 'PER', 'PER', 'PER', 'PER', 'O', 'O', 'O', 'O', 'O']
28
28


In [22]:
p_data = []
sentence_no = 0
new_df = []
for sentence, tags in zip(x['Sentence'].values.tolist(), x['Tag'].values.tolist()):
    words = sentence.split(" ")
    ind = [ i for i, word in enumerate(words) if word == "'s" or word == "'t" or word == "'re" or word == "'d" or word == "'ll" or word == "'t" or word == "'ve" or word == "'m" or word == "n't"]
    labels = tags.split(",")
    sentence = " ".join(words)
    sentence = decontracted(sentence)
    tags = [ elem for i, elem in enumerate(labels) if i not in ind]
    stripped = sentence.translate(str.maketrans('', '', string.punctuation))
    stripped = stripped.split(" ")
    for word, label in zip(stripped, tags):
        p_data.append((sentence_no, word, label))
    sentence_no = sentence_no + 1

In [23]:
df = pd.DataFrame(p_data, columns=['Sentence #', 'Word', 'Tag'])

In [24]:
df.head()

,Sentence #,Word,Tag
0,0,EU,ORG
1,0,rejects,O
2,0,German,O
3,0,call,O
4,0,to,O


In [25]:
df.drop(df[df['Word'] == ""].index, axis=0, inplace=True)

In [26]:
df.head()

,Sentence #,Word,Tag
0,0,EU,ORG
1,0,rejects,O
2,0,German,O
3,0,call,O
4,0,to,O


In [27]:
g_test = df.groupby("Sentence #")
x = pd.DataFrame({"Sentence": g_test.apply(lambda sdf: " ".join(map(str,sdf.Word))),
                       "Tag": g_test.apply(lambda sdf: ",".join(map(str,sdf.Tag)))})

In [28]:
x.head()

,Sentence,Tag
Sentence #,,
0,EU rejects German call to boycott British lamb,"ORG,O,O,O,O,O,O,O"
1,Peter Blackburn BRUSSELS 19960822 The European Commission said on Thursday it disagreed with German advice to consumers to shun British lamb until scientists determine whether mad cow disease can be transmitted to sheep,"PER,PER,LOC,O,O,ORG,ORG,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
2,Germanys representative to the European Unions veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer,"LOC,O,O,O,ORG,ORG,O,O,PER,PER,O,O,O,O,O,O,O,O,O,O,O,LOC,O,O,O,O,O,O"
3,We dont support any such recommendation because we dont see any grounds for it the Commissions chief spokesman Nikolaus van der Pas told a news briefing,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,ORG,O,O,PER,PER,PER,PER,O,O,O,O"
4,He said further scientific study was required and if it was found that action was needed it should be taken by the European Union,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,ORG,ORG"


In [29]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_whole_word_mask=True)

In [30]:
class SentenceGetter(object):

    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]

    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [31]:
getter = SentenceGetter(df)
sentences = [[word[0] for word in sentence] for sentence in getter.sentences]
labels = [[s[1] for s in sentence] for sentence in getter.sentences]

In [32]:
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(str(word))
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [33]:
tokenized_texts_and_labels = [tokenize_and_preserve_labels(sent, labs) for sent, labs in zip(sentences, labels)]
#tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
#labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

In [34]:
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

In [35]:
tokenized_texts[0:50]

[['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'la', '##mb'],
 ['Peter',
  'Blackburn',
  'BR',
  '##US',
  '##SE',
  '##LS',
  '1996',
  '##0',
  '##8',
  '##22',
  'The',
  'European',
  'Commission',
  'said',
  'on',
  'Thursday',
  'it',
  'disagreed',
  'with',
  'German',
  'advice',
  'to',
  'consumers',
  'to',
  's',
  '##hun',
  'British',
  'la',
  '##mb',
  'until',
  'scientists',
  'determine',
  'whether',
  'mad',
  'cow',
  'disease',
  'can',
  'be',
  'transmitted',
  'to',
  'sheep'],
 ['Germany',
  '##s',
  'representative',
  'to',
  'the',
  'European',
  'Union',
  '##s',
  'veterinary',
  'committee',
  'Werner',
  'Z',
  '##wing',
  '##mann',
  'said',
  'on',
  'Wednesday',
  'consumers',
  'should',
  'buy',
  'sheep',
  '##me',
  '##at',
  'from',
  'countries',
  'other',
  'than',
  'Britain',
  'until',
  'the',
  'scientific',
  'advice',
  'was',
  'clearer'],
 ['We',
  'don',
  '##t',
  'support',
  'any',
  'such',
  'recommendati

In [36]:
for sentence in tokenized_texts:
    print(sentence)
    break

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'la', '##mb']


In [37]:
i = 0
sentence_no = 0
new_sentence = []
new_data = []
for sentence, label in zip(tokenized_texts, labels):
    new_tokens = []
    new_tags = []
    for token, tag in zip(sentence, label):
        if token.startswith("##"):
            new_tokens[-1] = new_tokens[-1] + token[2:]
        else:
            new_tokens.append(token)
            new_tags.append(tag)
    for new_token, new_tag in zip(new_tokens, new_tags):
        if new_token in string.punctuation:
            continue
        else:
            new_data.append((sentence_no, new_token, new_tag))
    sentence_no = sentence_no + 1

In [38]:
new_data[0:50]

[(0, 'EU', 'ORG'),
 (0, 'rejects', 'O'),
 (0, 'German', 'O'),
 (0, 'call', 'O'),
 (0, 'to', 'O'),
 (0, 'boycott', 'O'),
 (0, 'British', 'O'),
 (0, 'lamb', 'O'),
 (1, 'Peter', 'PER'),
 (1, 'Blackburn', 'PER'),
 (1, 'BRUSSELS', 'LOC'),
 (1, '19960822', 'O'),
 (1, 'The', 'O'),
 (1, 'European', 'ORG'),
 (1, 'Commission', 'ORG'),
 (1, 'said', 'O'),
 (1, 'on', 'O'),
 (1, 'Thursday', 'O'),
 (1, 'it', 'O'),
 (1, 'disagreed', 'O'),
 (1, 'with', 'O'),
 (1, 'German', 'O'),
 (1, 'advice', 'O'),
 (1, 'to', 'O'),
 (1, 'consumers', 'O'),
 (1, 'to', 'O'),
 (1, 'shun', 'O'),
 (1, 'British', 'O'),
 (1, 'lamb', 'O'),
 (1, 'until', 'O'),
 (1, 'scientists', 'O'),
 (1, 'determine', 'O'),
 (1, 'whether', 'O'),
 (1, 'mad', 'O'),
 (1, 'cow', 'O'),
 (1, 'disease', 'O'),
 (1, 'can', 'O'),
 (1, 'be', 'O'),
 (1, 'transmitted', 'O'),
 (1, 'to', 'O'),
 (1, 'sheep', 'O'),
 (2, 'Germanys', 'LOC'),
 (2, 'representative', 'O'),
 (2, 'to', 'O'),
 (2, 'the', 'O'),
 (2, 'European', 'ORG'),
 (2, 'Unions', 'ORG'),
 (2, 'vete

In [39]:
data = pd.DataFrame(new_data, columns=['sentence_no', 'Word', 'Tag'])

In [40]:
data.head()

,sentence_no,Word,Tag
0,0,EU,ORG
1,0,rejects,O
2,0,German,O
3,0,call,O
4,0,to,O


In [41]:
data.to_csv("../../data/conll03/new_conll_train_preprocessed_without_punctuation.csv")